In [1]:
import requests
import pandas as pd
import datetime
import time
import random
from bs4 import BeautifulSoup
import os

# 2009~2010抓取

In [ ]:
contract_id = {'棉花':"CF",'菜油':'OI','白糖':'SR','PTA':'TA','强麦':'WH','硬麦':'PM','早籼':'RI'}
date = datetime.datetime(2009,1,19)

while (date == datetime.datetime(2009,1,19)):
    
    url = "http://www.czce.com.cn/portal/exchange/jyxx/pm/pm" + date.strftime('%Y%m%d') + ".html"
    res = requests.get(url)
    
    while(res.status_code != 200):
        date = date + datetime.timedelta(days = 1)
        res = requests.get(url)
        time.sleep(random.randrange(2,5))
        
    soup = BeautifulSoup(res.content.decode('gb2312'),"lxml")
    
    temp = []
    data_table = []
    name_count = 0
    
    name = []
    for i in soup.select("table")[1].find_all('div'):
        name.append(i.get_text().split()[1])
    

    for i in range(2,len(soup.select("table"))):

        for table in soup.select("table")[i].find_all('td'):
            temp.append(table.get_text().strip())

            if(temp[0] == ""):
                temp.clear()
                data = pd.DataFrame(data_table)

                if (len(name[name_count]) < 3):
                    data.to_csv("../各交易所期貨持倉量/鄭商所/更新_分類/" + contract_id[name[name_count]] + "_" + date.strftime('%Y%m%d') + ".csv",index=None)
                else:
                    data.to_csv("../各交易所期貨持倉量/鄭商所/更新_分類/" + name[name_count] + "_" + date.strftime('%Y%m%d') + ".csv",index=None)

                name_count += 1
                data_table.clear()
                continue

            if (len(temp)>=10):            
                data_table.append(list(temp))
                temp.clear()
                
    date = date + datetime.timedelta(days = 1)

# 2010之後抓取

In [29]:
date = datetime.datetime(2015,10,30)

while (date <= datetime.datetime(2018,5,24)):
    splite_date = str(date).split()[0].split('-')
    
    #2010-08-25~2015-09-30的網址
#     url_excel = ("http://www.czce.com.cn/portal/exchange/%s/datatradeholding/%s.txt" % \
#                  (splite_date[0],splite_date[0]+splite_date[1]+splite_date[2]))

    url_excel = ("http://www.czce.com.cn/portal/DFSStaticFiles/Future/%s/%s/FutureDataHolding.txt" % \
    (splite_date[0],splite_date[0]+splite_date[1]+splite_date[2]))
    
    res = requests.get(url_excel)
    
    while(res.status_code != 200 and date < datetime.datetime.now()):
        date = date + datetime.timedelta(days = 1)    
        splite_date = str(date).split()[0].split('-')
    
        url_excel = ("http://www.czce.com.cn/portal/DFSStaticFiles/Future/%s/%s/FutureDataHolding.txt" % \
        (splite_date[0],splite_date[0]+splite_date[1]+splite_date[2]))
        
        res = requests.get(url_excel)
        time.sleep(random.randrange(2,5))

    with open("../各交易所期貨持倉量/鄭商所/更新/"+splite_date[0]+splite_date[1]+splite_date[2]+".txt",'wb') as file:
        file.write(res.content)
        file.close()
        
    time.sleep(random.randrange(1,3)) 
    date = date + datetime.timedelta(days = 1)

# 資料分開

In [38]:
def find_zh(name_id):
    zh = ""
    for i in name_id:
        if (ord(i) > 255):
            zh += i
    if(zh == ""):
        return name_id
    
    return zh

DATA_DIR = '../各交易所期貨持倉量/鄭商所/更新/'
contract_id = {'棉花':"CF",'菜油':'OI','白糖':'SR','PTA':'TA','强麦':'WH','硬麦':'PM','早籼':'RI','甲醇':'MA',\
               '普麦':'PM','玻璃':'FG','菜粕':'RM','粳稻':'JR','菜籽':'RS','煤':'ZC','动力煤':'ZC',\
               '晚籼':'LR','硅铁':'SF','锰硅':'SM','苹果':'AP','棉纱':'CY'}
values = []

for filename in os.listdir(DATA_DIR):
    
    count = 0
    date = filename.split('.txt')[0]
    
#     if(datetime.datetime.strptime(date,"%Y%m%d") <= datetime.datetime(2017,12,22)):
#         continue
    
#     if(datetime.datetime.strptime(date,"%Y%m%d") > datetime.datetime(2015,11,10)):
#         break
        
    data = pd.read_table(DATA_DIR + filename,engine='python',sep="|",skiprows=[0,1,2],header=None)
    name = pd.read_table(DATA_DIR + filename,engine='python',sep="|",skiprows=[0,1],header=None,index_col=False)

    
    for i in range(len(data)):
        if(data.loc[i][0].strip() == '合计'):
            
            values.append(list(data.loc[i]))
            df = pd.DataFrame(values)    
            
            if(count == 0):
                name_temp = name.iloc[0][0].split()[0].split("：")[1]
                name_id = find_zh(name_temp)
                
                df = df.drop(df.index[0])
                df.to_csv("../各交易所期貨持倉量/鄭商所/更新_分類/" + contract_id[name_id] + "_" + date + ".csv",index=None)
                count += 1
                
                name_temp = name.iloc[i+2][0].split()[0].split("：")[1]
                name_id = find_zh(name_temp)

            elif(len(name_id) <= 4):
                df = df.drop(df.index[[0]])
                df.to_csv("../各交易所期貨持倉量/鄭商所/更新_分類/" + contract_id[name_id] + "_" + date + ".csv",index=None)
                
            else:
                df = df.drop(df.index[[0]])
                df.to_csv("../各交易所期貨持倉量/鄭商所/更新_分類/" + name_id + "_" + date + ".csv",index=None)
                                       
            if( i+2 <= len(data)):
                name_temp = name.iloc[i+2][0].split()[0].split("：")[1]
                name_id = find_zh(name_temp)

            values.clear()

        else:
            values.append(list(data.loc[i]))

KeyboardInterrupt: 

In [33]:
name_temp = name.iloc[0][0].split()[0].split("：")[1]

In [35]:
find_zh(name_temp)

'苹果'

In [37]:
for i in name_temp:
    if (ord(i) > 255):
        print(i)

苹
果
